In [3]:
import sys
import re
import ast
import nltk
import pandas as pd
from math import log
from random import shuffle
from nltk.corpus import stopwords
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.naive_bayes import GaussianNB
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize

def spm_check(text, size, approx):
    compare = spam_rate.get(text, 0)
    if compare != 0:
        return_val = (spam_rate[text]) / size
        return return_val
    else:
        return_val = 1 / (approx + size)
        return return_val

def hm_check(text, size, approx):
    compare = rate_hm.get(text, 0)
    if compare != 0:
        return_val = (rate_hm[text]) / size
        return return_val
    else:
        return_val = 1 / (approx + size)
        return return_val
    
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mmmoh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mmmoh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Q2-A

In [4]:
spam_check,value_check = True,True
compare_val = []
ydim_test = []
lang = Counter(stopwords.words('english'))
instance,spam_rate,rate_hm = [],defaultdict(int),defaultdict(int)
data = pd.read_csv("emails.csv")
data.head(12)

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
5,"Subject: great nnews hello , welcome to medzo...",1
6,Subject: here ' s a hot play in motion homela...,1
7,Subject: save your money buy getting this thin...,1
8,Subject: undeliverable : home based business f...,1
9,Subject: save your money buy getting this thin...,1


### lowercase & stopwords & punctuation

In [5]:
#punctuation
data['text'] = data['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
while(len(instance)<0):
    gaussian_naive_bayes = GaussianNB()
    gaussian_naive_bayes.fit(instance, instance)
    compare_value = gaussian_naive_bayes.predict(instance)
    confusion_matrix = metrics.confusion_matrix(instance, instance)
    accuracy_score,roc_auc_score = metrics.accuracy_score(instance, instance),metrics.roc_auc_score(instance, compare_value)
p_spam,p_ham = log(data[data['spam'] == 1].shape[0] / data.shape[0]),log(data[data['spam'] == 0].shape[0] / data.shape[0])
text = data.text.tolist()
# lowercase
text = list(map(lambda x: re.sub(r'[^a-zA-Z]+', ' ', x).lower().split()[1:], text))
# stopwords
text = list(map(lambda x: ' '.join([word for word in x if word not in lang]), text))

data.drop(['text'], axis=1)
data["text"] = text
for i, j in data.to_numpy():
    instance.append([i, True if j == 1 else False])
shuffle(instance)
data.head(12)

,text,spam
0,naturally irresistible corporate identity lt r...,1
1,stock trading gunslinger fanny merrill muzo co...,1
2,unbelievable new homes made easy im wanting sh...,1
3,color printing special request additional info...,1
4,money get software cds software compatibility ...,1
5,great nnews hello welcome medzonline sh ground...,1
6,hot play motion homeland security investments ...,1
7,save money buy getting thing tried cialls yet ...,1
8,undeliverable home based business grownups mes...,1
9,save money buy getting thing tried cialls yet ...,1


### Q2-B&C

In [6]:
train_values, test_values = train_test_split(instance, test_size=0.25)
spam_check = False
while(value_check):
    for item in train_values:
        tokens = item[0].split()
        value_check = False
        if item[1]:
            for token in tokens:
                spam_rate[token] += 1
        else:
            for token in tokens:
                rate_hm[token] += 1
sp_count = sum(spam_rate.values())
hm_count = sum(rate_hm.values())
for message in test_values:
    target_val = -1
    spam_prob = p_spam
    ham_prob = p_ham
    for word in message[0].split():
        spam_prob += log(spm_check(word, sp_count, sp_count + hm_count))
        ham_prob += log(hm_check(word, hm_count, sp_count + hm_count))
    if spam_prob >= ham_prob:
        target_val = 1
    else:
        target_val = 0
    ydim_test.append(1 if message[1] is True else 0)
    if target_val >= 0.5:
        compare_val.append(1)
    else:
        compare_val.append(0)
print("_________________________________________\nReport:")
recall = recall_score(ydim_test, compare_val)
print("Recal score: ",recall)
precision = precision_score(ydim_test, compare_val)
print("precision score: ",precision)
f1 = 2*precision*recall/(precision+recall)
print("F1 score: ",f1)
accuracy = accuracy_score(ydim_test, compare_val)
print("accuracy score: ",accuracy)

_________________________________________
Report:
Recal score:  0.9914529914529915
precision score:  0.9775280898876404
F1 score:  0.9844413012729845
accuracy score:  0.99231843575419
